In [1]:
# Import packages
import pandas as pd
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')
from sklearn.ensemble import RandomForestRegressor
from IPython.display import clear_output

# Set working path
path = '/Users/martinbogaert/Desktop/NBA Data Analysis/nba_ai_awards/'

print('MAKE SURE YOU CHANGE THE WEEK VARIABLE')

MAKE SURE YOU CHANGE THE WEEK VARIABLE


In [2]:
week = 17 # CHANGE THIS
date = datetime.now().strftime('%d %B %Y')
time = datetime.now().strftime('%H:%M:%S')
print(date + ' ; ' + time)

13 February 2023 ; 18:10:19


In [3]:
features = ['VORP','PTS','WS','OWS','FG','MP','OBPM','PER','BPM','Seed']
model = RandomForestRegressor(n_estimators = 100, max_features = 'auto', min_samples_leaf = 2)

# Minimum minutes per game
mp = 10
# Propotion of maximum games played
gp = 1/3

In [4]:
# Load training data
train_data = pd.read_csv(path + 'Algorithm/smoy/smoy_data.csv')

# Load to-be-predicted data
data = pd.read_csv(path + f'Algorithm/weekly data/week_{week}.csv') # Load up-to-date data
data = data.groupby('smoy').get_group(1)

# Load weekly info
with open(path + f'Algorithm/weekly data/week_{week}.txt', 'r') as file:
            date1 = file.readline().splitlines()[0][7:]
            date2 = file.readline().splitlines()[0][7:]
            n = int(file.readline()[12:])
            
season_progress = n / 15
data.head()

,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,FG%,...,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A,roy,smoy
0,A.J. Green,SG,23,MIL,25,0,8.8,1.5,3.4,0.440,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,1,1
1,Bobby Portis,PF,27,MIL,47,14,26.8,6.0,12.0,0.499,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,1
3,George Hill,PG,36,MIL,35,0,19.1,1.7,3.8,0.447,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,1
7,Joe Ingles,SF,35,MIL,23,0,22.2,2.1,5.4,0.384,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,1
8,Jordan Nwora,SF,24,MIL,38,3,15.7,2.0,5.2,0.386,...,3.02,114.35,111.43,2.92,2.82,114.33,111.58,2.74,0,1


In [5]:
# Set training data
train_data = train_data[train_data['Year'] < 2020] # DELETE THIS LINE
X_train = train_data[features]
y_train = train_data['Share']

# Set minimum requirements (games and minutes played)
data = data[data['MP'] >= mp] # Minimum of minutes per game
data = data[data['G'] > season_progress * gp] # At least 1/3rd of max games played

pred = []
for step in range(0, 10) :
    print('Model ' + str(step+1) + '/10 ...')
    clear_output(wait = True)
    # Fit Machine Learning model
    model.fit(X_train, y_train)

    # Predict shares of test data
    pred.append(model.predict(data[features]))

Model 10/10 ...


In [6]:
# Assemble results DataFrame
res = data.assign(Share = [np.array(pred)[:,i].mean() for i in range(0, len(data))])
res = res.sort_values('Share', ascending = False) # Sort values by predictions
res['Rank'] = list(range(1, len(res)+1))
res['week'] = len(res) * [week] # Add week

res = res.reset_index(drop = True)[['Player', 'MP', 'G', 'GS', 'Tm', 'PER', 'VORP', 'WS', 'PTS', 'Seed', 'OBPM', 'OWS', 'Share']]
res.head(5)

,Player,MP,G,GS,Tm,PER,VORP,WS,PTS,Seed,OBPM,OWS,Share
0,Christian Wood,27.9,46,17,DAL,20.7,2.139130,6.417391,17.6,4,2.0,3.386957,0.366463
1,Malcolm Brogdon,24.9,48,0,BOS,18.2,2.391667,6.491667,14.3,1,2.4,4.100000,0.289343
2,Bobby Portis,26.8,47,14,MIL,18.6,1.744681,6.804255,14.4,2,1.6,3.314894,0.145886
3,Grant Williams,27.6,56,20,BOS,10.9,0.732143,5.271429,8.7,1,-1.5,2.635714,0.077511
4,Bruce Brown,28.7,56,26,DEN,13.1,1.025000,4.392857,11.2,1,-0.8,1.903571,0.077497


In [7]:
res.to_csv(path + f'Results/smoy/results_week_{week}.csv', index = None)